In [21]:
from langchain_ollama import OllamaLLM
from langchain.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser 

import os
from dotenv import load_dotenv
load_dotenv()

import warnings 
warnings.filterwarnings("ignore")

import subprocess
model_name = "gemma3:4b"
subprocess.run(["ollama", "pull", model_name], check=True)

os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

In [22]:
llm = OllamaLLM(model = model_name)
llm.invoke("What are differences between a aphla and a beta male")

'Okay, let\'s break down the differences between Alpha and Beta males, as these terms have gained significant traction online, particularly within online communities like Reddit\'s r/masculinity. It\'s important to note that these are *social constructs* and not scientifically defined categories. They\'ve emerged from online discussions and self-identification, and their definitions are often debated and sometimes overly simplistic.\n\n**1. The "Alpha Male" Concept (Often Misunderstood)**\n\n* **Origin:** The "alpha male" concept comes from observing wolf packs. It\'s based on the idea that wolves establish dominance hierarchies within their packs.\n* **Traditional (and often problematic) Interpretation:** Originally, the "alpha male" was meant to represent a confident, assertive, dominant, and successful man who takes charge and leads.  This interpretation has been heavily promoted by self-help gurus and online communities, often with a focus on aggression, competition, and a need to 

In [23]:
parser = StrOutputParser()

In [27]:
chain = llm|parser
prompt_by_llm = chain.invoke("Give me a prompt that i can use for making a llm call to always give a structured and easy to understand output. Only return prompt and no extra explainations")
prompt_by_llm

"Respond to the following requests with a structured output, using the following format:\n\n**Question:** [User's Question]\n\n**Answer:**\n*   **Summary:** [Concise summary of the answer]\n*   **Key Points:**\n    *   [Point 1]\n    *   [Point 2]\n    *   [Point 3]\n    *   ...\n*   **Details:** [Expand on key points with relevant supporting information, if applicable]\n*   **Example(s):** [Provide an example to illustrate the concept]\n*   **Source(s):** [List any sources used to formulate the answer – this can be a brief citation or a link if appropriate.]\n"

In [28]:
prompt = ChatPromptTemplate.from_messages([
    ("system",prompt_by_llm),
    ("user",'{question}')
])

chain = prompt|llm|parser
chain.invoke("What are ways of prompting an llm")

'**Question:** What are ways of prompting an LLM?\n\n**Answer:**\n*   **Summary:** Prompting an LLM effectively involves crafting specific instructions and providing context to guide its output. Techniques range from simple instructions to complex, multi-turn interactions. The goal is to elicit the desired response format, accuracy, and style.\n\n*   **Key Points:**\n    *   **Clear Instructions:**  LLMs respond best to explicit instructions about the desired task, format, and length of the response.\n    *   **Context Provision:** Providing background information or context helps the LLM understand the scope and purpose of the request.\n    *   **Role-Playing:** Assigning the LLM a specific role (e.g., "Act as a marketing expert...") can dramatically improve the quality and relevance of its output.\n    *   **Few-Shot Learning:**  Including a few examples of desired input-output pairs within the prompt demonstrates the desired behavior.\n    *   **Chain-of-Thought Prompting:** Encoura

In [29]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

# Creating an empty dictionary for storing the session id'set
store = {}

def get_session_history(session_id : str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_chat_message_history = RunnableWithMessageHistory(llm,get_session_history)

config = {"configurable":{"session_id":'chat1'}}

chain = with_chat_message_history|parser

chain.invoke([HumanMessage('Why are pineapples pink?')], config=config)

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


'That\'s a fantastic question, and it gets to a really interesting quirk of pineapple development! Here\'s the breakdown of why some pineapples are pink:\n\n**It\'s a genetic mutation!**\n\n* **The \'Rose\' Pineapple:** The pink color you see in some pineapples, like the \'Rose\' pineapple, isn\'t a natural characteristic of the fruit itself. It\'s a result of a recessive genetic mutation.\n* **Anthocyanins:** This mutation causes the pineapple to produce high levels of anthocyanins – these are the same pigments that create the red and purple colors in berries like blueberries and blackberries.\n* **Lack of Chlorophyll:**  The mutation also inhibits the production of chlorophyll, which is what gives most pineapples their green color.  Without enough chlorophyll, the flesh appears pale and then takes on the reddish-pink hue from the anthocyanins.\n\n**Here\'s the key:**\n\n* **Dominant Green is Normal:**  The *typical* pineapple has a dominant gene that produces plenty of chlorophyll, r

In [30]:
chain.invoke("what was my last question", config=config)

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


'You asked: “What was my last question?” 😊 \n\nIs that correct?'

In [31]:
chain.invoke("hi im rahul", config=config)

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


'Hi Rahul! It’s nice to meet you. 😊 \n\nHow’s your day going so far? Is there anything you’d like to chat about, or were you just saying hello?'

In [32]:
chain.invoke("what is my name", config=config)

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


"I do not have access to personal information, including your name. As an AI, I have no memory of past conversations and cannot recall details about you. \n\nTo tell me your name, you'll need to tell me! 😊"

In [1]:
import subprocess
subprocess.run("ollama list",check=True,  capture_output=True, text=True)

CompletedProcess(args='ollama list', returncode=0, stdout='NAME                        ID              SIZE      MODIFIED       \ngemma3:4b                   a2af6cc3eb7f    3.3 GB    38 seconds ago    \ncodellama:7b                8fdf8f752f6e    3.8 GB    24 minutes ago    \nqwen2.5-coder:7b            dae161e27b0e    4.7 GB    26 minutes ago    \nphi3:latest                 4f2222927938    2.2 GB    29 minutes ago    \nembeddinggemma:300m         85462619ee72    621 MB    15 hours ago      \ngemma3:270m                 e7d36fb2c3b3    291 MB    17 hours ago      \nall-minilm:22m              1b226e2802db    45 MB     18 hours ago      \nqwen3:4b                    e55aed6fe643    2.5 GB    9 days ago        \nmxbai-embed-large:latest    468836162de7    669 MB    10 days ago       \ndeepseek-r1:7b              755ced02ce7b    4.7 GB    13 days ago       \ndeepseek-r1:latest          6995872bfe4c    5.2 GB    13 days ago       \n', stderr='')